#IIC-2433 Minería de Datos UC

* Versiones de librerías, python 3.8.10
* pandas 1.5.3
* tensorflow/keras 2.12.0

In [3]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer

# 1. Carga de la base de datos

En esta tarea se trabajará con un dataset de vinos obtenido de Kaggle:
Cárguelo, léalo y muéstrelo.

https://www.kaggle.com/datasets/yasserh/wine-quality-dataset

In [4]:
dframe = pd.read_csv("WineQT.csv", encoding = "ISO-8859-1")
dframe.shape

(1143, 13)

In [5]:
dframe

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,Id
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,0
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5,1
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5,2
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6,3
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1138,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6,1592
1139,6.8,0.620,0.08,1.9,0.068,28.0,38.0,0.99651,3.42,0.82,9.5,6,1593
1140,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5,1594
1141,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6,1595


# 2. Preprocesamiento

Realice el preprocesamiento que considere adecuado para este *dataset* y argumente todas sus decisiones.

In [6]:
 #Chequeamos si hay algun valor nulo en el df, y si hay eliminamos la fila
if dframe.isnull().values.any():
    dframe.dropna()

In [7]:
# Calcular el rango intercuartílico
Q1 = dframe.quantile(0.25)
Q3 = dframe.quantile(0.75)
IQR = Q3 - Q1

# Definir los límites inferior y superior para detectar outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filtrar los outliers
filtered_df = dframe[~((dframe < lower_bound) | (dframe > upper_bound)).any(axis=1)]
filtered_df


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,Id
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,0
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5,1
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5,2
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6,3
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1138,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6,1592
1139,6.8,0.620,0.08,1.9,0.068,28.0,38.0,0.99651,3.42,0.82,9.5,6,1593
1140,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5,1594
1141,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6,1595


In [8]:
X = filtered_df.drop('quality', axis=1)
y = filtered_df['quality']

# 3. Preguntas iniciales


Deberá encontrar la mejor combinación de hiperparámetros que le permita obtener el modelo con mejor *accuracy*.

A priori, ¿cuáles cree que serán las mejores combinaciones de hiperparámetros para este problema en particular? ¿Por qué?

# 3. Modelos

Ocupando la librería **keras** de tensorflow, construya redes neuronales multicapas variando los siguientes hiperparámetros: función de activación, cantidad de neuronas por cada capa, cantidad de capas, optimizador y cantidad de épocas.

Intente encontrar un modelo que obtenga un 66.6% de *accuracy*. ¿Lo logró? ¿por qué?

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
def create_model(optimizer='adam', activation='relu', neurons=10, hidden_layers=1):
    model = Sequential()
    model.add(Dense(neurons, activation=activation, input_shape=(X_train.shape[1],)))
    for i in range(hidden_layers):
        model.add(Dense(neurons, activation=activation))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer=optimizer, loss='mse', metrics=['accuracy'])
    return model


In [11]:
model = KerasClassifier(build_fn=create_model, verbose=0)

C:\Users\javie\AppData\Local\Temp\ipykernel_25300\2566461152.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, verbose=0)


In [12]:
param_grid = {
    'epochs': [50, 100, 150],
    'optimizer': ['SGD', 'RMSprop', 'Adagrad', 'Adam'],
    'activation': ['relu', 'tanh', 'sigmoid'],
    'neurons': [5, 10, 15],
    'hidden_layers': [1, 2, 3]
}

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

# Resumen de los resultados
print("Mejor: %f usando %s" % (grid_result.best_score_, grid_result.best_params_))

Mejor: 0.445300 usando {'activation': 'relu', 'epochs': 50, 'hidden_layers': 1, 'neurons': 5, 'optimizer': 'SGD'}


Una vez obtenido el mejor clasificador, ¿cuáles son sus errores más comunes? ¿cómo se podrían mejorar?

# 4. Preguntas finales

¿Qué resultados obtuvo? Se habrá dado cuenta que encontrar el mejor modelo no es lo mismo que buscar los mejores hiperparámetros unilateralmente. ¿Cuáles combinaciones funcionaron mejor y cuáles peor? ¿Por qué cree que fue así? Argumente.